In [1]:
import os
import pyarrow.parquet as pq
import pandas as pd

In [2]:
df = pd.DataFrame()

In [3]:
parquet_folder = 'Dataset'
output_csv_file = 'process_output.csv'

In [4]:
def convert_parquet_to_csv(parquet_folder, output_csv_file):
    # Initialize an empty list to store the dataframes
    dfs = []

    # Iterate over the Parquet files in the folder, in a specific order
    for i in range(1, 3): 
        file_name = f"yellow_tripdata_2016-{i}.parquet"
        file_path = os.path.join(parquet_folder, file_name)
        if os.path.exists(file_path):
            # Read the Parquet file
            table = pq.read_table(file_path)
            # Convert to Pandas DataFrame
            df = table.to_pandas()
            # Append to the list of DataFrames
            dfs.append(df)

    # Concatenate the DataFrames
    combined_df = pd.concat(dfs, ignore_index=True)

    # Write to CSV file
    combined_df.to_csv(output_csv_file, index=False)



In [5]:
convert_parquet_to_csv(parquet_folder, output_csv_file)